# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb2e094d550>
├── 'a' --> tensor([[ 1.0598,  0.1059, -0.1148],
│                   [-1.8377,  0.1462, -0.5735]])
└── 'x' --> <FastTreeValue 0x7fb2e094d130>
    └── 'c' --> tensor([[-0.8571, -0.9131,  0.8059, -0.6705],
                        [-2.3284,  0.8214,  2.7057,  0.3792],
                        [-0.7817,  0.6273, -0.0736, -0.2197]])

In [4]:
t.a

tensor([[ 1.0598,  0.1059, -0.1148],
        [-1.8377,  0.1462, -0.5735]])

In [5]:
%timeit t.a

71.7 ns ± 1.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb2e094d550>
├── 'a' --> tensor([[-1.6285, -2.3936,  1.2108],
│                   [ 0.2074, -0.4971,  0.4403]])
└── 'x' --> <FastTreeValue 0x7fb2e094d130>
    └── 'c' --> tensor([[-0.8571, -0.9131,  0.8059, -0.6705],
                        [-2.3284,  0.8214,  2.7057,  0.3792],
                        [-0.7817,  0.6273, -0.0736, -0.2197]])

In [7]:
%timeit t.a = new_value

70.1 ns ± 2.56 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0598,  0.1059, -0.1148],
               [-1.8377,  0.1462, -0.5735]]),
    x: Batch(
           c: tensor([[-0.8571, -0.9131,  0.8059, -0.6705],
                      [-2.3284,  0.8214,  2.7057,  0.3792],
                      [-0.7817,  0.6273, -0.0736, -0.2197]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0598,  0.1059, -0.1148],
        [-1.8377,  0.1462, -0.5735]])

In [11]:
%timeit b.a

64.3 ns ± 0.211 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3778, -0.0337, -1.7540],
               [ 0.0477,  2.3110, -1.4531]]),
    x: Batch(
           c: tensor([[-0.8571, -0.9131,  0.8059, -0.6705],
                      [-2.3284,  0.8214,  2.7057,  0.3792],
                      [-0.7817,  0.6273, -0.0736, -0.2197]]),
       ),
)

In [13]:
%timeit b.a = new_value

594 ns ± 9.08 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

991 ns ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.5 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

282 µs ± 5.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

266 µs ± 6.89 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb2e094deb0>
├── 'a' --> tensor([[[ 1.0598,  0.1059, -0.1148],
│                    [-1.8377,  0.1462, -0.5735]],
│           
│                   [[ 1.0598,  0.1059, -0.1148],
│                    [-1.8377,  0.1462, -0.5735]],
│           
│                   [[ 1.0598,  0.1059, -0.1148],
│                    [-1.8377,  0.1462, -0.5735]],
│           
│                   [[ 1.0598,  0.1059, -0.1148],
│                    [-1.8377,  0.1462, -0.5735]],
│           
│                   [[ 1.0598,  0.1059, -0.1148],
│                    [-1.8377,  0.1462, -0.5735]],
│           
│                   [[ 1.0598,  0.1059, -0.1148],
│                    [-1.8377,  0.1462, -0.5735]],
│           
│                   [[ 1.0598,  0.1059, -0.1148],
│                    [-1.8377,  0.1462, -0.5735]],
│           
│                   [[ 1.0598,  0.1059, -0.1148],
│                    [-1.8377,  0.1462, -0.5735]]])
└── 'x' --> <FastTreeValue 0x7fb23559af40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44.7 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb23557f4c0>
├── 'a' --> tensor([[ 1.0598,  0.1059, -0.1148],
│                   [-1.8377,  0.1462, -0.5735],
│                   [ 1.0598,  0.1059, -0.1148],
│                   [-1.8377,  0.1462, -0.5735],
│                   [ 1.0598,  0.1059, -0.1148],
│                   [-1.8377,  0.1462, -0.5735],
│                   [ 1.0598,  0.1059, -0.1148],
│                   [-1.8377,  0.1462, -0.5735],
│                   [ 1.0598,  0.1059, -0.1148],
│                   [-1.8377,  0.1462, -0.5735],
│                   [ 1.0598,  0.1059, -0.1148],
│                   [-1.8377,  0.1462, -0.5735],
│                   [ 1.0598,  0.1059, -0.1148],
│                   [-1.8377,  0.1462, -0.5735],
│                   [ 1.0598,  0.1059, -0.1148],
│                   [-1.8377,  0.1462, -0.5735]])
└── 'x' --> <FastTreeValue 0x7fb23559bf40>
    └── 'c' --> tensor([[-0.8571, -0.9131,  0.8059, -0.6705],
                        [-2.3284,  0.8214,  2.7057,  0.3792],
                 

In [23]:
%timeit t_cat(trees)

41.5 µs ± 927 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

75.8 µs ± 995 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.0598,  0.1059, -0.1148],
                [-1.8377,  0.1462, -0.5735]],
       
               [[ 1.0598,  0.1059, -0.1148],
                [-1.8377,  0.1462, -0.5735]],
       
               [[ 1.0598,  0.1059, -0.1148],
                [-1.8377,  0.1462, -0.5735]],
       
               [[ 1.0598,  0.1059, -0.1148],
                [-1.8377,  0.1462, -0.5735]],
       
               [[ 1.0598,  0.1059, -0.1148],
                [-1.8377,  0.1462, -0.5735]],
       
               [[ 1.0598,  0.1059, -0.1148],
                [-1.8377,  0.1462, -0.5735]],
       
               [[ 1.0598,  0.1059, -0.1148],
                [-1.8377,  0.1462, -0.5735]],
       
               [[ 1.0598,  0.1059, -0.1148],
                [-1.8377,  0.1462, -0.5735]]]),
    x: Batch(
           c: tensor([[[-0.8571, -0.9131,  0.8059, -0.6705],
                       [-2.3284,  0.8214,  2.7057,  0.3792],
                       [-0.7817,  0.6273, -0.0736, -0.2197]],
         

In [26]:
%timeit Batch.stack(batches)

97.8 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.0598,  0.1059, -0.1148],
               [-1.8377,  0.1462, -0.5735],
               [ 1.0598,  0.1059, -0.1148],
               [-1.8377,  0.1462, -0.5735],
               [ 1.0598,  0.1059, -0.1148],
               [-1.8377,  0.1462, -0.5735],
               [ 1.0598,  0.1059, -0.1148],
               [-1.8377,  0.1462, -0.5735],
               [ 1.0598,  0.1059, -0.1148],
               [-1.8377,  0.1462, -0.5735],
               [ 1.0598,  0.1059, -0.1148],
               [-1.8377,  0.1462, -0.5735],
               [ 1.0598,  0.1059, -0.1148],
               [-1.8377,  0.1462, -0.5735],
               [ 1.0598,  0.1059, -0.1148],
               [-1.8377,  0.1462, -0.5735]]),
    x: Batch(
           c: tensor([[-0.8571, -0.9131,  0.8059, -0.6705],
                      [-2.3284,  0.8214,  2.7057,  0.3792],
                      [-0.7817,  0.6273, -0.0736, -0.2197],
                      [-0.8571, -0.9131,  0.8059, -0.6705],
                      [-2.3284,  

In [28]:
%timeit Batch.cat(batches)

177 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

579 µs ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
